In [1]:
import pandas as pd
import numpy as np
import jieba
import time
import csv
# 停词表
import requests
import json
import plotly.graph_objs as go
import plotly
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = 'all'
import matplotlib.pyplot as plt
import os

# 情感词典构建,感叹号两倍，分句

In [73]:
# 分词，返回List
jieba.load_userdict('70000-dict.txt')
def read_lines(filename):
    fp = open(filename, 'r',encoding='utf-8')
    lines = []
    for line in fp.readlines():
        line = line.strip()
        line = line.encode('utf-8').decode("utf-8")
        lines.append(line)
    fp.close()
    return lines
def segmentation(sentence):
    seg_list = jieba.cut(sentence)
    seg_result = []
    for w in seg_list:
        seg_result.append(w)
    return seg_result
# 去除停用词
def del_stopwords(seg_sent):
    stopwords = read_lines("Sentiment_dict/emotion_dict/stop_words.txt")  # 读取停用词表
    new_sent = []  # 去除停用词后的句子
    for word in seg_sent:
        if word in stopwords:
            continue
        else:
            new_sent.append(word)
    return new_sent
def match(word, sentiment_value):
    if word in mostdict:
        sentiment_value *= 2.0
    elif word in verydict:
        sentiment_value *= 1.75
    elif word in moredict:
        sentiment_value *= 1.5
    elif word in ishdict:
        sentiment_value *= 1.2
    elif word in insufficientdict:
        sentiment_value *= 0.5
    elif word in inversedict:
        #print "inversedict", word
        sentiment_value *= -1
    return sentiment_value

# 句子切分
def cut_sentence(words):
    words = words.encode('utf-8').decode('utf-8')
    start = 0
    i = 0
    token = 'meaningless'
    sents = []
    punt_list = ',.!?;~，。！？；～… '.encode('utf-8').decode('utf-8')
    #print "punc_list", punt_list
    for word in words:
        #print "word", word
        if word not in punt_list:  # 如果不是标点符号
            #print "word1", word
            i += 1
            token = list(words[start:i + 2]).pop()
            #print "token:", token
        elif word in punt_list and token in punt_list:  # 处理省略号
            #print "word2", word
            i += 1
            token = list(words[start:i + 2]).pop()
            #print "token:", token
        else:
            #print "word3", word
            sents.append(words[start:i + 1])  # 断句
            start = i + 1
            i += 1
    if start < len(words):  # 处理最后的部分
        sents.append(words[start:])
    return sents[:-1]

def single_score(sentence):
    single_review_senti_score = []
    cuted_review=cut_sentence(sentence)
    for sent in cuted_review:
        seg_sent = segmentation(sent)   # 分词
        seg_sent = del_stopwords(seg_sent)[:]
        i = 0    # 记录扫描到的词的位置
        s = 0    # 记录上一个情感词的位置
        emotion_value = 0    # 记录该分句中的积极情感得分

        for word in seg_sent:   # 逐词分析
            # word限定了三个条件，所以只是关键词
            #print word
            if word in emotion_keys:
                a = emotion_dict.get(word)
                for w in seg_sent[s:i]:
                    a = match(w, a)
                emotion_value+=a
                #print "poscount:", poscount
                s = i + 1  # 记录情感词的位置变化

            #如果是感叹号，表示已经到本句句尾
            elif word == "！".encode('utf-8').decode("utf-8") or word == "!".encode('utf-8').decode('utf-8'):
                # 有问题
                emotion_value*=2
                break
            i += 1
        #print "poscount,negcount", poscount, negcount
        single_review_senti_score.append(emotion_value)   # 对得分做最后处理

    result=np.array(single_review_senti_score).sum()
    result = round(result, 1)
    return result

# 读取语料计算情感得分

In [2]:
# 情感词典构建

# 分词，返回List
jieba.load_userdict('70000-dict.txt')
def read_lines(filename):
    fp = open(filename, 'r',encoding='utf-8')
    lines = []
    for line in fp.readlines():
        line = line.strip()
        line = line.encode('utf-8').decode("utf-8")
        lines.append(line)
    fp.close()
    return lines
## 程度词和否定词文件
mostdict = read_lines('Sentiment_dict/degree_dict/most.txt')   # 权值为2
verydict = read_lines('Sentiment_dict/degree_dict/very.txt')   # 权值为1.5
moredict = read_lines('Sentiment_dict/degree_dict/more.txt')   # 权值为1.25
ishdict = read_lines('Sentiment_dict/degree_dict/ish.txt')   # 权值为0.5
insufficientdict = read_lines('Sentiment_dict/degree_dict/insufficiently.txt')  # 权值为0.25
inversedict = read_lines('Sentiment_dict/degree_dict/inverse.txt')  # 权值为-1
def segmentation(sentence):
    seg_list = jieba.cut(sentence)
    seg_result = []
    for w in seg_list:
        seg_result.append(w)
    return seg_result
# 去除停用词
def del_stopwords(seg_sent):
    stopwords = read_lines("Sentiment_dict/emotion_dict/stop_words.txt")  # 读取停用词表
    new_sent = []  # 去除停用词后的句子
    for word in seg_sent:
        if word in stopwords:
            continue
        else:
            new_sent.append(word)
    return new_sent
def match(word, sentiment_value):
    if word in mostdict:
        sentiment_value *= 2.0
    elif word in verydict:
        sentiment_value *= 1.75
    elif word in moredict:
        sentiment_value *= 1.5
    elif word in ishdict:
        sentiment_value *= 1.2
    elif word in insufficientdict:
        sentiment_value *= 0.5
    elif word in inversedict:
        #print "inversedict", word
        sentiment_value *= -1
    return sentiment_value


def get_keys():
    emotion=pd.read_csv("Sentiment_dict/emotion_dict/BosonNLP_sentiment_score.txt",sep='\s+',header=None)
#     CON=emotion.iloc[:,0].isin([',','.','!','?',';','~','，','。','！','？','；','～','… ','[',']'])
#     emotion=emotion[~CON]
    
    emotion_dict=dict(zip(emotion.iloc[:,0],emotion.iloc[:,1]))
    emotion_keys=emotion.iloc[:,0].values.tolist()
    return emotion_keys,emotion_dict

emotion_keys,emotion_dict=get_keys()

def single_score(sentence):
    seg_sent = del_stopwords(segmentation(sentence))
    
    i = 0    # 记录扫描到的词的位置
    s = 0    # 记录上一个情感词的位置
    emotion_value =[]   # 记录该分句中的积极情感得分
    
    for word in seg_sent:   # 逐词分析
        # word限定了三个条件，所以只是关键词
        
        if word in emotion_keys:
#             print('情感词',word)
            a = emotion_dict.get(word)
#             print('初始分',a)
            for w in seg_sent[s:i]:
#                 print('权重',w)
                a = match(w, a)
#                 print('加权后',a)
            # 记录该词的情感值 
            emotion_value.append(a)
            s = i + 1  # 记录情感词的位置变化
#         if word == "！" or word == "!":
#             try:
#                 emotion_value[-1]*=2
#             except:
#                 print('error')
        i += 1
    result = round(sum(emotion_value), 1)
    return result,len(emotion_value)

def cal_score(original_name='缺判断缺评分',name='缺判断'):
    # 输入original为读入文件，name为读出文件
    import time
    start_time=time.time()
    
    _ = open("D:\\IDEA\\情感分析论文\\论文数据\\" + name + '.csv',
             'w',
             encoding='utf-8')
    _.close()
    print(name + '.csv已创建')

    # 创建写入器
    csvfile = open("D:\\IDEA\\情感分析论文\\论文数据\\" + name + '.csv',
                   'a+',
                   newline='',
                   encoding='utf-8')
    # a+表示以追加模式写入，如果用w会覆盖掉原来的数据
    csv_write = csv.writer(csvfile)

    # 读取并写入
    print(name + '.csv正在写入')
    f = open('D:\\IDEA\\情感分析论文\\论文数据\\'+original_name+'.csv',
             encoding='utf-8')
    csv_read = csv.reader(f)
    for i,line in enumerate(csv_read):
        score,length=single_score(line[0])
        print(i,score)
        _ = csv_write.writerow([score,length])
    csvfile.close()
    f.close()
    print(name + '.csv写入完成')
    end_time=time.time()
    print('%.8s'%(end_time-start_time))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\RUBYEE~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.747 seconds.
Prefix dict has been built successfully.


# 测试一些词

In [9]:
words='《济南战“疫”》：今日报道了济南市三例治愈出院的患者，祝福他们！加油！[加油][加油][加油] '

seg_sent=segmentation(words)   # 分
sen=del_stopwords(seg_sent)
# 分词
sen
# 得分
single_score(words)
emotion_keys,emotion_dict=get_keys()
emotion_dict.get('2019')

['《',
 '济南',
 '战',
 '“',
 '疫',
 '”',
 '》',
 '：',
 '今日',
 '报道',
 '了',
 '济南市',
 '三例',
 '治愈',
 '出院',
 '患者',
 '，',
 '祝福',
 '！',
 '加油',
 '！',
 '[',
 '加油',
 ']',
 '[',
 '加油',
 ']',
 '[',
 '加油',
 ']',
 ' ']

(30.5, 29)

3.37612173454

In [88]:
del_stopwords(segmentation('这两天看到太多黑暗面，红十字的不作为，一线人员的可怜，疫情的不断爆发，心太累了，听到这首歌，心灵得到了短暂的慰籍，加油，武汉，还是有正能量人在支撑我们'))
emotion_dict.get('允悲')

['这两天',
 '看到',
 '太多',
 '黑暗面',
 '，',
 '红十字',
 '的不',
 '，',
 '一线',
 '人员',
 '可怜',
 '，',
 '疫情',
 '不断',
 '爆发',
 '，',
 '心',
 '太累',
 '了',
 '，',
 '听到',
 '这首歌',
 '，',
 '心灵',
 '得到',
 '了',
 '短暂',
 '慰籍',
 '，',
 '加油',
 '，',
 '武汉',
 '，',
 '正能量',
 '人在',
 '支撑']